In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('IMDB Dataset.csv')

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
df['user_id'] = np.random.randint(1, 1001, df.shape[0])
df['movie_id'] = np.random.randint(1, 1001, df.shape[0])

df['rating'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

df.head()

,review,sentiment,user_id,movie_id,rating
0,One of the other reviewers has mentioned that ...,positive,746,981,1
1,A wonderful little production. <br /><br />The...,positive,311,181,1
2,I thought this was a wonderful way to spend ti...,positive,272,308,1
3,Basically there's a family where a little boy ...,negative,949,902,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,313,384,1


In [3]:
user_item_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')

user_item_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_item_matrix.fillna(0))

user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def get_user_recommendations(user_id, num_recommendations=5):
    similarity_scores = user_similarity_df[user_id]
    
    user_ratings = user_item_matrix.loc[user_id]
    
    weighted_ratings = user_item_matrix.T.dot(similarity_scores) / similarity_scores.sum()
    
    recommendations = weighted_ratings[~user_ratings.isna()]
    
    top_recommendations = recommendations.sort_values(ascending=False).head(num_recommendations)
    
    return top_recommendations

recommendations = get_user_recommendations(user_id=1, num_recommendations=5)
print(recommendations)


movie_id
11   NaN
19   NaN
53   NaN
58   NaN
73   NaN
dtype: float64
